# This file is used for simulating artificial data under Heston model. Do not run the file, or it will change the result as presented in the report.

In [1]:
import numpy as np
from functools import partial
from scipy.integrate import quad
import pandas as pd
rng = np.random.default_rng()

### The following option pricing code for the Heston model is from this GitHub repository.https://github.com/cantaro86/Financial-Models-Numerical-Methods

In [2]:
def Heston_pdf(i, t, v0, mu, theta, sigma, kappa, rho):
    """
    Heston density by Fourier inversion.
    """
    cf_H_b_good = partial(cf_Heston_good, t=t, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho )
    return Gil_Pelaez_pdf(i, cf_H_b_good, np.inf)

def Q1(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the stock numeraire.
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( (np.exp(-u*k*1j) / (u*1j)) * 
                                  cf(u-1j) / cf(-1.0000000000001j) )  
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=2000 )[0]

def Q2(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the money market numeraire
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( np.exp(-u*k*1j) /(u*1j) * cf(u) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=2000 )[0]

def cf_Heston_good(u, t, v0, mu, kappa, theta, sigma, rho):
    """
    Heston characteristic function as proposed by Schoutens (2004)
    """
    xi = kappa - sigma*rho*u*1j
    d = np.sqrt( xi**2 + sigma**2 * (u**2 + 1j*u) )
    g1 = (xi+d)/(xi-d)
    g2 = 1/g1
    cf = np.exp( 1j*u*mu*t + (kappa*theta)/(sigma**2) * ( (xi-d)*t - 2*np.log( (1-g2*np.exp(-d*t))/(1-g2) ))\
              + (v0/sigma**2)*(xi-d) * (1-np.exp(-d*t))/(1-g2*np.exp(-d*t)) )
    return cf

In [3]:
r = 0                                              # drift
rho = -0.8                                         # correlation coefficient
kappa = 1                                          # mean reversion coefficient
theta = 0.2                                        # long-term mean of the variance
sigma = 0.5                                        # (Vol of Vol) - Volatility of instantaneous variance
T = 0.0951864535768645                             # Terminal time
K = 7250                                           # Stike  
v0 = 0.6                                           # spot variance
S0 = 8276.43                                       # spot stock price 
k = np.log(K/S0)                                   # log moneyness


cf_H_b_good = partial(cf_Heston_good, t=T, v0=v0, mu=r, theta=theta, sigma=sigma, kappa=kappa,rho = rho ) 

# %%time
limit_max = 2000      # right limit in the integration                
call = S0 * Q1(k, cf_H_b_good, limit_max) - K * np.exp(-r*T) * Q2(k, cf_H_b_good, limit_max)
print("Heston Fourier inversion call price: ", call)
print("-----------------------------------")

Heston Fourier inversion call price:  1361.583790572182
-----------------------------------


In [4]:
A = {
    't': 30/365,   # Terminal time
    'v0': 0.6,      # spot variance
    'mu': 0,       # drift
    'theta': 0.2,  # long-term mean of the variance
    'sigma': 0.5,# (Vol of Vol) - Volatility of instantaneous variance
    'kappa': 1,# mean reversion coefficient
    'rho': -0.8     # correlation coefficient
}

In [5]:
B = {
    't': 30/365,   # Terminal time
    'v0': 0.6,      # spot variance
    'mu': 0,       # drift
    'theta': 0.2,  # long-term mean of the variance
    'sigma': 1,# (Vol of Vol) - Volatility of instantaneous variance
    'kappa': 1,# mean reversion coefficient
    'rho': -0.4     # correlation coefficient
}

In [6]:
C = {
    't': 30/365,   # Terminal time
    'v0': 0.6,      # spot variance
    'mu': 0,       # drift
    'theta': 0.04,  # long-term mean of the variance
    'sigma': 1,# (Vol of Vol) - Volatility of instantaneous variance
    'kappa': 5,# mean reversion coefficient
    'rho': -0.4     # correlation coefficient
}

In [7]:
D = {
    't': 30/365,   # Terminal time
    'v0': 0.04,      # spot variance
    'mu': 0,       # drift
    'theta': 0.04,  # long-term mean of the variance
    'sigma': 0.3,# (Vol of Vol) - Volatility of instantaneous variance
    'kappa': 1.5,# mean reversion coefficient
    'rho': -0.7     # correlation coefficient
}

In [8]:
def get_data(strike_data,params,s0 = 4100):
    call = []
    put = []
    limit_max = 3000
    for K in strike_data:
        k = np.log(K/s0)
        cf_H_b_good = partial(cf_Heston_good, **params)
        c = s0 * Q1(k, cf_H_b_good, limit_max) - K * np.exp(-params["mu"]*params["t"]) * Q2(k, cf_H_b_good, limit_max)
        p = round(c-s0+K,2)
        call.append(round(c,2))
        put.append(p)
    dic_call = {"contractSymbol":["call"]*len(strike_data),"strike":strike_data,"lastPrice":call}
    dic_put = {"contractSymbol":["put"]*len(strike_data),"strike":strike_data,"lastPrice":put}
    call = pd.DataFrame(dic_call)
    put = pd.DataFrame(dic_put)
    return call,put

In [9]:
def simulate_bid_ask(data,tick=1):
    bid = rng.geometric(0.8,size = len(data)) 
    ask = rng.geometric(0.8,size = len(data)) 
    a = []
    b = []
    for i,p in enumerate(data["lastPrice"].values):
        if p>=5:
            a.append(p+tick*ask[i])
            b.append(p-tick*bid[i])
        else:
            a.append(p+ask[i]*(0.05*p))
            b.append(p-bid[i]*(0.05*p))
    data["bid"] = b
    data["ask"] = a
    data["volume"] = np.ones(len(data))
    return data

In [10]:
strike1 = np.arange(2000,3000,100) # deep OTM put
strike2 =  np.arange(3000,4000,50) # OTM put
strike3 = np.arange(4000,4200,5) # ITM
strike4 = np.arange(4200,5000,50) # OTM call
strike5 = np.arange(5000,7500,100)# deep OTM call
strike = np.concatenate((strike1,strike2,strike3,strike4,strike5),axis = None)
strike.shape

(111,)

# Wider range strike

In [12]:
from strike_data import strike
strike.shape

(326,)